In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [17]:
#combine a lot of steps: get the beautiful soup function called on the webpage
soup = BeautifulSoup(
    requests.get(
        "https://www.sports-reference.com/cbb/seasons/men/2021-school-stats.html", 
        headers={"User-Agent": "Chrome"}).content,
    'html.parser')


In [25]:
# Find the table element by its class name
table = soup.find('table', {'class': 'per_match_toggle sortable stats_table', 'id': 'basic_school_stats'})


In [26]:
# Extract the table headers
headers = []
for th in table.find_all('th'):
    headers.append(th.text.strip())

    #Since the headers occur every 20 teams, and have, there's like 1000. 
headers=headers[13:50]
subheaders=headers[0:12]

In [27]:
# Extract the table rows
rows = []
for tr in table.find_all('tr')[1:]:
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    rows.append(row)
rows

[[],
 ['Abilene Christian\xa0NCAA',
  '29',
  '24',
  '5',
  '.828',
  '6.27',
  '-6.37',
  '',
  '13',
  '2',
  '',
  '13',
  '0',
  '',
  '5',
  '4',
  '',
  '2196',
  '1753',
  '',
  '1160',
  '786',
  '1752',
  '.449',
  '221',
  '643',
  '.344',
  '403',
  '594',
  '.678',
  '329',
  '1047',
  '513',
  '282',
  '86',
  '394',
  '554'],
 ['Air Force',
  '25',
  '5',
  '20',
  '.200',
  '-12.98',
  '0.22',
  '',
  '3',
  '17',
  '',
  '3',
  '8',
  '',
  '1',
  '10',
  '',
  '1468',
  '1798',
  '',
  '1000',
  '522',
  '1170',
  '.446',
  '174',
  '527',
  '.330',
  '250',
  '361',
  '.693',
  '110',
  '580',
  '308',
  '177',
  '62',
  '361',
  '415'],
 ['Akron',
  '23',
  '15',
  '8',
  '.652',
  '1.85',
  '-1.96',
  '',
  '12',
  '6',
  '',
  '9',
  '1',
  '',
  '5',
  '5',
  '',
  '1798',
  '1660',
  '',
  '930',
  '628',
  '1415',
  '.444',
  '217',
  '652',
  '.333',
  '325',
  '429',
  '.758',
  '262',
  '895',
  '305',
  '113',
  '84',
  '271',
  '413'],
 ['Alabama\xa0NCAA',

In [28]:
# Create a pandas dataframe from the headers and rows
df = pd.DataFrame(rows, columns=headers,)

# Print the dataframe
df.dropna(axis = 0, how = 'all', inplace = True)
df.insert(0, 'Rank', range(1, 1+len(df)))

# Drop the index column
df = df.reset_index(drop=True)
df
#might need to specify conf and all that jazz. Like subheadings.

,Rank,School,G,W,L,W-L%,SRS,SOS,,W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian NCAA,29,24,5,.828,6.27,-6.37,,13,...,403,594,.678,329,1047,513,282,86,394,554
1,2,Air Force,25,5,20,.200,-12.98,0.22,,3,...,250,361,.693,110,580,308,177,62,361,415
2,3,Akron,23,15,8,.652,1.85,-1.96,,12,...,325,429,.758,262,895,305,113,84,271,413
3,4,Alabama NCAA,33,26,7,.788,19.58,10.01,,16,...,433,612,.708,408,1343,464,280,143,458,603
4,5,Alabama A&M,15,6,9,.400,-18.86,-14.99,,4,...,173,276,.627,160,577,164,82,54,229,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,351,Wright State,24,18,6,.750,6.89,-7.65,,16,...,368,508,.724,284,1015,369,142,87,314,328
351,352,Wyoming,25,14,11,.560,-1.23,-2.31,,5,...,366,509,.719,203,816,351,145,54,281,468
352,353,Xavier,21,13,8,.619,11.09,6.47,,6,...,231,333,.694,200,792,369,145,85,235,325
353,354,Yale,0,0,0,,,,,,...,,,,,,,,,,


In [7]:
# define lambda function to check if "NCAA" is present in the string
is_ncaa = lambda x: "Yes" if "NCAA" in x else "No"

# apply the lambda function to each row in the "School" column and create a new "Tournament" column
df['Tournament'] = df['School'].apply(is_ncaa)

# insert the "Tournament" column at position 2 (index 1)
df.insert(1, 'Tournament', df.pop('Tournament'))

# print the dataframe
df

,Rank,Tournament,School,G,W,L,W-L%,SRS,SOS,,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Yes,Abilene Christian NCAA,29,24,5,.828,6.27,-6.37,,...,403,594,.678,329,1047,513,282,86,394,554
1,2,No,Air Force,25,5,20,.200,-12.98,0.22,,...,250,361,.693,110,580,308,177,62,361,415
2,3,No,Akron,23,15,8,.652,1.85,-1.96,,...,325,429,.758,262,895,305,113,84,271,413
3,4,Yes,Alabama NCAA,33,26,7,.788,19.58,10.01,,...,433,612,.708,408,1343,464,280,143,458,603
4,5,No,Alabama A&M,15,6,9,.400,-18.86,-14.99,,...,173,276,.627,160,577,164,82,54,229,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,351,No,Wright State,24,18,6,.750,6.89,-7.65,,...,368,508,.724,284,1015,369,142,87,314,328
351,352,No,Wyoming,25,14,11,.560,-1.23,-2.31,,...,366,509,.719,203,816,351,145,54,281,468
352,353,No,Xavier,21,13,8,.619,11.09,6.47,,...,231,333,.694,200,792,369,145,85,235,325
353,354,No,Yale,0,0,0,,,,,...,,,,,,,,,,


In [8]:
df.to_csv('march_mad_data.csv', index=False)

In [9]:
list(df.columns)

['Rank',
 'Tournament',
 'School',
 'G',
 'W',
 'L',
 'W-L%',
 'SRS',
 'SOS',
 '',
 'W',
 'L',
 '',
 'W',
 'L',
 '',
 'W',
 'L',
 '',
 'Tm.',
 'Opp.',
 '',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF']